In [83]:
import torch
import pygame
import numpy as np
import os

# Directory for saving frames

frame_directory = "frames"
os.makedirs(frame_directory, exist_ok=True)

# Ensure that CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Simulation parameters
width, height = 400, 400
Nx, Ny = 100, 100
dx, dy = 1.0 / Nx, 1.0 / Ny
dt = 0.01  # Time step
viscosity = 0.02

x_center_1, y_center_1 = 0.5, 0.5  # Center of the 1st Gaussian
x_center_2, y_center_2 = 1.5, 0.5  # Center of the 2nd Gaussian

sigma_1 = 0.10  # Standard deviation of the 1st Gaussian
sigma_2 = 0.20  # Standard deviation of the 2nd Gaussian

magnitude = 200  # Magnitude of the temperature increase

# Initialize the temperature array with two Gaussian distributions

X, Y = torch.meshgrid(torch.linspace(0, 2, Nx), torch.linspace(0, 1, Ny))


In [84]:

gaussian_field_1 = magnitude * torch.exp(-((X - x_center_1)**2 + (Y - y_center_1)**2) / (2 * sigma_1**2))
gaussian_field_2 = magnitude * torch.exp(-((X - x_center_2)**2 + (Y - y_center_2)**2) / (2 * sigma_2**2))
# plt.matshow(gaussian_field_1
#             + gaussian_field_2)
# plt.show()
T = (200 + gaussian_field_1 + gaussian_field_2).unsqueeze(0).unsqueeze(1).to(device)


In [85]:
T.shape


torch.Size([1, 1, 100, 100])

In [86]:
T_pad = torch.nn.functional.pad(T, (1, 1), mode="constant")

In [87]:
T_pad.shape

torch.Size([1, 1, 100, 102])

In [88]:
kernel = torch.tensor([[0, 1, 0],
                        [1, -4, 1],
                        [0, 1, 0]], dtype=torch.float32, device=device).unsqueeze(0).unsqueeze(1)

In [89]:
torch.clone()

TypeError: clone() missing 1 required positional arguments: "input"

In [90]:
kernel.shape

torch.Size([1, 1, 3, 3])

In [91]:
class HeatEquationSolver:
    def __init__(self, T, kernel, dt, dx, dy, viscosity):
        self.T = T
        self.kernel = kernel
        self.dt = dt
        self.dx = dx
        self.dy = dy
        self.viscosity = viscosity

    def step(self):
        # Pad the temperature array
        T_pad = torch.clone(self.T)

        # Compute the Laplacian
        laplacian = torch.nn.functional.conv2d(T_pad, self.kernel, stride=1, padding = 1)

        # Compute the diffusion term
        diffusion = self.viscosity * laplacian[1:-1, 1:-1]

        # Compute the new temperature
        self.T[1:-1, 1:-1] += self.dt * diffusion

        # Enforce boundary conditions
        self.T[:, :, 0, :] = self.T[:, :, 1, :]
        self.T[:, :, -1, :] = self.T[:, :, -2, :]
        self.T[:, :, :, 0] = self.T[:, :, :, 1]
        self.T[:, :, :, -1] = self.T[:, :, :, -2]
        
    def run(self, num_steps):
        for i in range(num_steps):
            self.step()

In [92]:
# Pygame setup
pygame.init()
screen = pygame.display.set_mode((width, height))
clock = pygame.time.Clock()

running = True
frame_number = 0
update_skip = 10
MULT = 4
solver = HeatEquationSolver(T, kernel, dt, dx, dy, viscosity)
last_T = solver.T
while running:

    
    solver.step()
    print(torch.allclose(solver.T, last_T))
    # Transfer data to CPU for visualization and convert to NumPy
    # T_cpu = solver.T.repeat(1, 3, 1, 1)

    T_cpu = T.cpu().detach().squeeze(1).squeeze(0).numpy()
    print(T_cpu.shape, T_cpu.min())
    T_color = T_cpu - np.min(T_cpu)
    T_color = T_color / np.max(T_color)
    T_color = np.uint8(255 * T_color)
    # print(T_color.shape, np.max(T_color))

    # T_cpu = solver.T.cpu().squeeze(1).numpy()
    # Visualization
    for i in range(Nx):
        for j in range(Ny):
            color = T_color[i, j]
            # print(T_color[i, j])
            # color = min(0, max(0, int(255 * color)))
            pygame.draw.rect(screen, (color, color, color), (i * width / Nx, j * height / Ny, width / Nx, height / Ny))

    # pygame.surfarray.blit_array(screen, T_cpu)
    pygame.display.flip()
    clock.tick(60)
    
    # Save frame
    pygame.image.save(screen, os.path.join(frame_directory, f"frame_{frame_number:04d}.png"))
    frame_number += 1
    # print(frame_number)
    # Check for Pygame quit events
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

pygame.quit()


True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100, 100) 200.0
True
(100,